# Lab-06-02 ZOO

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

print(tf.__version__)

tf.random.set_seed(777)  # for reproducibility

2.3.0


## Data Load

In [15]:
df = pd.read_csv('data-04-zoo.csv')
df = df[18:]
list_column = [i for i in range(16)] + ['label']
df.columns = list_column
df = df.reset_index()

col_trainX = [i for i in range(16)]
col_trainY = ['label']

df_trainX = df[col_trainX].astype(np.float32)
df_trainY = df[col_trainY]
df_trainY = df_trainY.astype(np.int64)

Make trainY one-hot encode format

In [17]:
nb_classes = 7
Y_one_hot = tf.one_hot(df_trainY, nb_classes)
print(f'X-shape {df_trainX.shape}\tY-shape {Y_one_hot.shape}')
Y_one_hot = tf.reshape(Y_one_hot, (101, 7))
print(f'X-shape {df_trainX.shape}\tY-shape {Y_one_hot.shape}')

X-shape (101, 16)	Y-shape (101, 1, 7)
X-shape (101, 16)	Y-shape (101, 7)


Make to dataset

## Hypothesis

In [9]:
W = tf.Variable(tf.random.uniform((16, 7), -10, 10), name = 'weight')
b = tf.Variable(tf.random.uniform((1,7), -10, 10), name = 'bias')
variables  = [W, b]
def linearRegression_fn(X):
    return tf.matmul(X, W) +  b

def hypothesis_fn(X):
    return tf.nn.softmax(linearRegression_fn(X))

# hypothesis(df_trainX)

## Loss

In [19]:
def loss_fn(X, Y):
    return tf.reduce_mean(tf.keras.losses.categorical_crossentropy(Y, linearRegression_fn(X), from_logits=True))
loss_fn(df_trainX, Y_one_hot)

<tf.Tensor: shape=(), dtype=float32, numpy=42.86525>

## Grad

In [20]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = loss_fn(X, Y)
        grads = tape.gradient(loss, variables)
    return grads

# grad_fn(df_trainX, Y_one_hot)

## Train

In [26]:
learning_rate = 0.2
verbose = 100
epoch = 1000
def train_fn(X, Y, learning_rate = learning_rate, epoch = epoch, verbose = verbose):
    optimizer = tf.keras.optimizers.SGD(learning_rate)
    for step in tqdm(range(epoch)):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if step % verbose == 0 :
            print(f'step - {step}\t loss_mean - {loss_fn(X, Y).numpy()} ')
train_fn(df_trainX, Y_one_hot)

  0%|          | 0/1000 [00:00<?, ?it/s]

step - 0	 loss_mean - 0.26602903008461 
step - 100	 loss_mean - 0.2257828265428543 
step - 200	 loss_mean - 0.1923115998506546 
step - 300	 loss_mean - 0.16403643786907196 
step - 400	 loss_mean - 0.14025065302848816 
step - 500	 loss_mean - 0.12081892788410187 
step - 600	 loss_mean - 0.10534373670816422 
step - 700	 loss_mean - 0.09302973002195358 
step - 800	 loss_mean - 0.08308032900094986 
step - 900	 loss_mean - 0.07489807903766632 


## Check

In [32]:
def accuracy_fn(X, Y):
    prediction = hypothesis_fn(X)
    prediction = tf.argmax(prediction, 1)
    correct_prediction = tf.equal(prediction, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy
accuracy_fn(df_trainX, Y_one_hot)
    

<tf.Tensor: shape=(), dtype=float32, numpy=0.980198>

## make as a class

In [36]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tf.Variable(tf.random.normal((16, nb_classes)), name='weight')
        self.b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(logits), axis=1))        
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(X, Y)
            grads = tape.gradient(cost, self.variables)            
            return grads
    
    def fit(self, X, Y, epochs=2000, verbose=500):
        optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.cost_fn(X, Y).numpy()))
            
model = softmax_classifer(nb_classes)
model.fit(df_trainX, Y_one_hot)
# model.predict(df_trainX[1])

Loss at epoch 1: 9.198975
Loss at epoch 500: 0.186254
Loss at epoch 1000: 0.101434
Loss at epoch 1500: 0.070091
Loss at epoch 2000: 0.053641
